In [1]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

True

## Firebase Credentials

In [2]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URI'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URI'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
  }


In [3]:
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred,{
    'storageBucket': 'coffee-recommendation-engine.firebasestorage.app',
     'databaseURL': 'https://coffee-recommendation-engine.firebaseio.com/'
})

In [4]:
bucket = storage.bucket()


## Upload Data

In [6]:
image_folder_path = './products/images/'


In [9]:
products_collection = db.reference('products')


In [10]:
df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)


,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [11]:
def upload_image(bucket, image_path):
    image_name = image_path.split('/')[-1]
    blob = bucket.blob(f'product_images/{image_name}')
    # Upload image
    blob.upload_from_filename(image_path)
    # Make the image publicly accessible and get its URL
    blob.make_public()
    return blob.public_url

In [12]:
# for index, row in df.iterrows():
#     print(index, row['name'])
    
#     image_path = os.path.join(image_folder_path,row['image_path'])
    
#     image_url = upload_image(bucket,image_path)
#     product_data = row.to_dict()
#     product_data.pop('image_path')
#     product_data['image_url']= image_url
    
#     # Add to Firestore
#     products_collection.push().set(product_data)

for index, row in df.iterrows():
    try:
        print(f"Processing row {index}: {row['name']}")
        
        # Prepare the image path and upload the image
        image_path = os.path.join(image_folder_path,row['image_path'])
        image_url = upload_image(bucket, image_path)
        
        if not image_url:
            print(f"Skipping Firestore upload for row {index} due to image upload failure.")
            continue  # Skip Firestore entry if image upload failed
        
        # Prepare product data for Firestore
        product_data = row.to_dict()
        product_data.pop('image_path')  # Remove local image path
        product_data['image_url'] = image_url  # Add Firebase Storage URL
        
        # Add data to Firestore
        products_collection.push().set(product_data)    
        print(f"Product {row['name']} added to Firestore.")
        
    except Exception as e:
        print(f"Error processing row {index} ({row['name']}): {e}")    

Processing row 0: Cappuccino
Product Cappuccino added to Firestore.
Processing row 1: Jumbo Savory Scone
Product Jumbo Savory Scone added to Firestore.
Processing row 2: Latte
Product Latte added to Firestore.
Processing row 3: Chocolate Chip Biscotti
Product Chocolate Chip Biscotti added to Firestore.
Processing row 4: Espresso shot
Product Espresso shot added to Firestore.
Processing row 5: Hazelnut Biscotti
Product Hazelnut Biscotti added to Firestore.
Processing row 6: Chocolate Croissant
Product Chocolate Croissant added to Firestore.
Processing row 7: Dark chocolate
Product Dark chocolate added to Firestore.
Processing row 8: Cranberry Scone
Product Cranberry Scone added to Firestore.
Processing row 9: Croissant
Product Croissant added to Firestore.
Processing row 10: Almond Croissant
Product Almond Croissant added to Firestore.
Processing row 11: Ginger Biscotti
Product Ginger Biscotti added to Firestore.
Processing row 12: Oatmeal Scone
Product Oatmeal Scone added to Firestore.